In [ ]:
!pip install matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import csv

def load_from_file(region):
    g = nx.DiGraph()
    
    with open('nodelist_{}.csv'.format(region), 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            g.add_nodes_from(row)
    
    edges = list()
    with open('edgelist_{}.csv'.format(region), 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            edges.append((row['Source'], row['Target']))
    g.add_edges_from(edges)
    
    return g

In [ ]:
graph = load_from_file('netherlands')

print("Analyzing network of {} nodes and {} edges".format(graph.order(), graph.size()))

print("Out-degree centrality distribution: ")
odc = nx.out_degree_centrality(graph)

plt.figure()
plt.yscale('log')
plt.ylabel('No. of nodes (log)')
plt.xlabel('Out-degree centrality')
plt.hist(odc.values(), bins=20)
plt.show()

In [ ]:
import operator

key_actors = {k: round(v, 4) for k,v in odc.items() if v > 0.01}
sorted_k = sorted(key_actors.items(), key=operator.itemgetter(1), reverse=True)

print(sorted_k)

In [ ]:
DB_HOST = '89.145.163.87'
DB_PORT = 5432
DB_USERNAME = 'group11'
DB_PASSWORD = 'tsw2020'
DB_NAME = 'tweets'

In [ ]:
import psycopg2
import pandas as pd

def db_to_pandas(query):
    """ Query database to a Pandas DF"""
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME, user=DB_USERNAME, password=DB_PASSWORD)
    cur = conn.cursor()
    
    df = pd.read_sql_query(query, conn)
    print(df.info)
    
    cur.close()
    conn.close()
    
    return df

In [ ]:
key_actor_tweets = pd.DataFrame()
for actor in sorted_k:

    query = "SELECT tweet_body->'text' as text FROM tweet WHERE tweet_body->'user'->>'id_str' = '{}';".format(actor[0])
    df = db_to_pandas(query)
    for row in df.iterrows():
        key_actor_tweets['tweet_id'] = actor[0]
        key_actor_tweets['similarity_score'] = actor[1]
        key_actor_tweets['text'] = row

    print(key_actor_tweets.shape)